<h1>OPERATING SYSTEM-COCSC09</h1>
<h2>CPU SCHEDULING PROJECT</h2>
<br>
<p> <b>Round-Robin</b> CPU scheduling algorithm has been implemented on the following dataset: 
        <a href="http://gwa.ewi.tudelft.nl/datasets/gwa-t-12-bitbrains"> Dataset </a>
</p>
<p> After studying the parameters given in the dataset, it is assumed that all the processes arrive 
    at the same time i.e 0<sup>th</sup> second and the parameter "Timestamp" denotes the CPU Burst-time
    of that process in miliseconds. The scheduling algorithm has been implemented on one of the CSV files,
    however it can be implemented on other files also just by changing the path of the file in "pd.read_csv"
    function.
</p>


In [1]:
#Importing Libraries
import numpy as np
import pandas as pd

In [2]:
# Reading the CSV file- "2013-7/4.csv"
# Change path here 
# NOTE: If path of the file is changed, change the value of n in the main function accordingly
df=pd.read_csv('D:/Scheduling Project/rnd/rnd/2013-7/4.csv',header=0,sep=";+\t")

C:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
# First five data-points
df.head()

,Timestamp [ms],CPU cores,CPU capacity provisioned [MHZ],CPU usage [MHZ],CPU usage [%],Memory capacity provisioned [KB],Memory usage [KB],Disk read throughput [KB/s],Disk write throughput [KB/s],Network received throughput [KB/s],Network transmitted throughput [KB/s]
0,1372629804,1,2599.999006,0.000000,0.0,1300480.0,50330.666667,0.0,4.933333,0.066667,0.066667
1,1372630104,1,2599.999006,5.199998,0.2,1300480.0,75495.200000,0.0,1.066667,0.200000,0.066667
2,1372630404,1,2599.999006,0.000000,0.0,1300480.0,36348.000000,0.0,0.000000,0.000000,0.000000
3,1372630704,1,2599.999006,5.199998,0.2,1300480.0,16776.000000,0.0,0.066667,0.266667,0.133333
4,1372631004,1,2599.999006,0.000000,0.0,1300480.0,44736.800000,0.0,0.000000,0.000000,0.000000


In [4]:
# Calculating some mean measures of the dataset
df.describe()

,Timestamp [ms],CPU cores,CPU capacity provisioned [MHZ],CPU usage [MHZ],CPU usage [%],Memory capacity provisioned [KB],Memory usage [KB],Disk read throughput [KB/s],Disk write throughput [KB/s],Network received throughput [KB/s],Network transmitted throughput [KB/s]
count,8.253000e+03,8253.0,8253.000000,8253.000000,8253.000000,8.253000e+03,8253.000000,8253.000000,8253.000000,8253.000000,8253.000000
mean,1.373920e+09,1.0,2599.999443,21.063282,0.810126,2.895928e+06,112822.091757,0.062547,4.674054,1.000413,2.211326
std,7.646588e+05,0.0,0.000156,13.536926,0.520651,1.250790e+06,66952.736177,3.769708,11.096060,4.048541,3.356133
min,1.372630e+09,1.0,2599.999006,0.000000,0.000000,1.300480e+06,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.373249e+09,1.0,2599.999398,10.399998,0.400000,1.347584e+06,58718.400000,0.000000,0.933333,0.066667,0.133333
50%,1.373868e+09,1.0,2599.999398,25.999994,1.000000,3.792486e+06,117439.200000,0.000000,4.800000,0.266667,1.266667
75%,1.374600e+09,1.0,2599.999398,31.199993,1.200000,4.059136e+06,153788.800000,0.000000,5.666667,0.533333,3.200000
max,1.375221e+09,1.0,2599.999691,313.733261,12.066667,4.059136e+06,757769.333333,318.600000,381.066667,221.666667,120.866667


In [5]:
# General Information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8253 entries, 0 to 8252
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Timestamp [ms]                         8253 non-null   int64  
 1   CPU cores                              8253 non-null   int64  
 2   CPU capacity provisioned [MHZ]         8253 non-null   float64
 3   CPU usage [MHZ]                        8253 non-null   float64
 4   CPU usage [%]                          8253 non-null   float64
 5   Memory capacity provisioned [KB]       8253 non-null   float64
 6   Memory usage [KB]                      8253 non-null   float64
 7   Disk read throughput [KB/s]            8253 non-null   float64
 8   Disk write throughput [KB/s]           8253 non-null   float64
 9   Network received throughput [KB/s]     8253 non-null   float64
 10  Network transmitted throughput [KB/s]  8253 non-null   float64
dtypes: f

In [6]:
# Function for finding the waiting time
def findWaitingTime(processes, n, bt,wt, quantum):
	rem_bt = [0] * n

	# Copy the burst time into rt[]
	for i in range(n):
		rem_bt[i] = bt[i]
	t = 0 # Current time

	# Keep traversing processes in round
	# robin manner until all of them are
	# not done.
	while(1):
		done = True

		# Traverse all processes one by
		# one repeatedly
		for i in range(n):
			
			# If burst time of a process is greater
			# than 0 then only need to process further
			if (rem_bt[i] > 0) :
				done = False # There is a pending process
				
				if (rem_bt[i] > quantum) :
				
					# Increase the value of t i.e. shows
					# how much time a process has been processed
					t += quantum

					# Decrease the burst_time of current
					# process by quantum
					rem_bt[i] -= quantum
				
				# If burst time is smaller than or equal
				# to quantum. Last cycle for this process
				else:
				
					# Increase the value of t i.e. shows
					# how much time a process has been processed
					t = t + rem_bt[i]

					# Waiting time is current time minus
					# time used by this process
					wt[i] = t - bt[i]

					# As the process gets fully executed
					# make its remaining burst time = 0
					rem_bt[i] = 0
				
		# If all processes are done
		if (done == True):
			break

In [7]:
# Function for finding the Turnaround time
def findTurnAroundTime(processes, n, bt, wt, tat):
	
	# Calculating turnaround time
	for i in range(n):
		tat[i] = bt[i] + wt[i]

In [8]:
#Function for finding the average-time
def findavgTime(processes, n, bt, quantum):
	wt = [0] * n
	tat = [0] * n

	# Function to find waiting time
	# of all processes
	findWaitingTime(processes, n, bt,wt, quantum)
	# Function to find turn around time
	# for all processes
	findTurnAroundTime(processes, n, bt,wt, tat)
	# Display processes along with all details
	print("Processes \tBurst Time[ms]\t\t Waiting","Time[ms]\t Turn-Around Time[ms]")
	total_wt = 0
	total_tat = 0
	for i in range(n):
		total_wt = total_wt + wt[i]
		total_tat = total_tat + tat[i]
		print(" ", i + 1, "\t\t", bt[i],"\t\t", wt[i], "\t\t", tat[i])
	print("\nAverage waiting time[ms] = %.5f "%(total_wt /n) )
	print("Average turn around time[ms] = %.5f "% (total_tat / n))

In [9]:
# Main Function for performing round-robin scheduing on the dataset
if __name__ =="__main__":
    #  8253 is the number of data-points in "2013-7/4.csv" 
	proc = list(range(0,8253))
	n = 8253
	# Burst time of all processes
	burst_time = []
	for x in df['Timestamp [ms]']:
		burst_time.append(x)
	# Time quantum
	quantum = 1000000000;
	findavgTime(proc, n, burst_time, quantum)
#Scroll to the bottom of the result to see average time result

Processes 	Burst Time[ms]		 Waiting Time[ms]	 Turn-Around Time[ms]
  1 		 1372629804 		 8252000000000 		 8253372629804
  2 		 1372630104 		 8252372629804 		 8253745259908
  3 		 1372630404 		 8252745259908 		 8254117890312
  4 		 1372630704 		 8253117890312 		 8254490521016
  5 		 1372631004 		 8253490521016 		 8254863152020
  6 		 1372631304 		 8253863152020 		 8255235783324
  7 		 1372631604 		 8254235783324 		 8255608414928
  8 		 1372631904 		 8254608414928 		 8255981046832
  9 		 1372632204 		 8254981046832 		 8256353679036
  10 		 1372632504 		 8255353679036 		 8256726311540
  11 		 1372632804 		 8255726311540 		 8257098944344
  12 		 1372633104 		 8256098944344 		 8257471577448
  13 		 1372633404 		 8256471577448 		 8257844210852
  14 		 1372633704 		 8256844210852 		 8258216844556
  15 		 1372634004 		 8257216844556 		 8258589478560
  16 		 1372634304 		 8257589478560 		 8258962112864
  17 		 1372634604 		 8257962112864 		 8259334747468
  18 		 1372634903 		 8258334747468 		 82

  241 		 1372701811 		 8341439757707 		 8342812459518
  242 		 1372702111 		 8341812459518 		 8343185161629
  243 		 1372702411 		 8342185161629 		 8343557864040
  244 		 1372702711 		 8342557864040 		 8343930566751
  245 		 1372703011 		 8342930566751 		 8344303269762
  246 		 1372703311 		 8343303269762 		 8344675973073
  247 		 1372703611 		 8343675973073 		 8345048676684
  248 		 1372703911 		 8344048676684 		 8345421380595
  249 		 1372704211 		 8344421380595 		 8345794084806
  250 		 1372704511 		 8344794084806 		 8346166789317
  251 		 1372704811 		 8345166789317 		 8346539494128
  252 		 1372705111 		 8345539494128 		 8346912199239
  253 		 1372705411 		 8345912199239 		 8347284904650
  254 		 1372705711 		 8346284904650 		 8347657610361
  255 		 1372706011 		 8346657610361 		 8348030316372
  256 		 1372706311 		 8347030316372 		 8348403022683
  257 		 1372706611 		 8347403022683 		 8348775729294
  258 		 1372706911 		 8347775729294 		 8349148436205
  259 		 1372707211 		 83481

  441 		 1372762124 		 8415986098250 		 8417358860374
  442 		 1372762424 		 8416358860374 		 8417731622798
  443 		 1372762724 		 8416731622798 		 8418104385522
  444 		 1372763024 		 8417104385522 		 8418477148546
  445 		 1372763324 		 8417477148546 		 8418849911870
  446 		 1372763624 		 8417849911870 		 8419222675494
  447 		 1372763924 		 8418222675494 		 8419595439418
  448 		 1372764224 		 8418595439418 		 8419968203642
  449 		 1372764524 		 8418968203642 		 8420340968166
  450 		 1372764824 		 8419340968166 		 8420713732990
  451 		 1372765124 		 8419713732990 		 8421086498114
  452 		 1372765424 		 8420086498114 		 8421459263538
  453 		 1372765724 		 8420459263538 		 8421832029262
  454 		 1372766024 		 8420832029262 		 8422204795286
  455 		 1372766324 		 8421204795286 		 8422577561610
  456 		 1372766624 		 8421577561610 		 8422950328234
  457 		 1372766924 		 8421950328234 		 8423323095158
  458 		 1372767224 		 8422323095158 		 8423695862382
  459 		 1372767524 		 84226

  865 		 1372889335 		 8574064143712 		 8575437033047
  866 		 1372889635 		 8574437033047 		 8575809922682
  867 		 1372889935 		 8574809922682 		 8576182812617
  868 		 1372890235 		 8575182812617 		 8576555702852
  869 		 1372890535 		 8575555702852 		 8576928593387
  870 		 1372890835 		 8575928593387 		 8577301484222
  871 		 1372891135 		 8576301484222 		 8577674375357
  872 		 1372891435 		 8576674375357 		 8578047266792
  873 		 1372891735 		 8577047266792 		 8578420158527
  874 		 1372892035 		 8577420158527 		 8578793050562
  875 		 1372892335 		 8577793050562 		 8579165942897
  876 		 1372892635 		 8578165942897 		 8579538835532
  877 		 1372892935 		 8578538835532 		 8579911728467
  878 		 1372893235 		 8578911728467 		 8580284621702
  879 		 1372893535 		 8579284621702 		 8580657515237
  880 		 1372893835 		 8579657515237 		 8581030409072
  881 		 1372894135 		 8580030409072 		 8581403303207
  882 		 1372894435 		 8580403303207 		 8581776197642
  883 		 1372894735 		 85807

  1146 		 1372973942 		 8678857884486 		 8680230858428
  1147 		 1372974242 		 8679230858428 		 8680603832670
  1148 		 1372974542 		 8679603832670 		 8680976807212
  1149 		 1372974842 		 8679976807212 		 8681349782054
  1150 		 1372975142 		 8680349782054 		 8681722757196
  1151 		 1372975442 		 8680722757196 		 8682095732638
  1152 		 1372975742 		 8681095732638 		 8682468708380
  1153 		 1372976042 		 8681468708380 		 8682841684422
  1154 		 1372976342 		 8681841684422 		 8683214660764
  1155 		 1372976642 		 8682214660764 		 8683587637406
  1156 		 1372976942 		 8682587637406 		 8683960614348
  1157 		 1372977242 		 8682960614348 		 8684333591590
  1158 		 1372977542 		 8683333591590 		 8684706569132
  1159 		 1372977842 		 8683706569132 		 8685079546974
  1160 		 1372978142 		 8684079546974 		 8685452525116
  1161 		 1372978442 		 8684452525116 		 8685825503558
  1162 		 1372978742 		 8684825503558 		 8686198482300
  1163 		 1372979042 		 8685198482300 		 8686571461342
  1164 		 

...

  7431 		 1374974836 		 11029258577062 		 11030633551898
  7432 		 1374975136 		 11029633551898 		 11031008527034
  7433 		 1374975436 		 11030008527034 		 11031383502470
  7434 		 1374975736 		 11030383502470 		 11031758478206
  7435 		 1374976036 		 11030758478206 		 11032133454242
  7436 		 1374976336 		 11031133454242 		 11032508430578
  7437 		 1374976636 		 11031508430578 		 11032883407214
  7438 		 1374976936 		 11031883407214 		 11033258384150
  7439 		 1374977236 		 11032258384150 		 11033633361386
  7440 		 1374977536 		 11032633361386 		 11034008338922
  7441 		 1374977836 		 11033008338922 		 11034383316758
  7442 		 1374978136 		 11033383316758 		 11034758294894
  7443 		 1374978436 		 11033758294894 		 11035133273330
  7444 		 1374978736 		 11034133273330 		 11035508252066
  7445 		 1374979036 		 11034508252066 		 11035883231102
  7446 		 1374979336 		 11034883231102 		 11036258210438
  7447 		 1374979636 		 11035258210438 		 11036633190074
  7448 		 1374979936 		 1103563

  7590 		 1375022540 		 11088883344590 		 11090258367130
  7591 		 1375022840 		 11089258367130 		 11090633389970
  7592 		 1375023140 		 11089633389970 		 11091008413110
  7593 		 1375023440 		 11090008413110 		 11091383436550
  7594 		 1375023740 		 11090383436550 		 11091758460290
  7595 		 1375024040 		 11090758460290 		 11092133484330
  7596 		 1375024340 		 11091133484330 		 11092508508670
  7597 		 1375024640 		 11091508508670 		 11092883533310
  7598 		 1375024940 		 11091883533310 		 11093258558250
  7599 		 1375025240 		 11092258558250 		 11093633583490
  7600 		 1375025540 		 11092633583490 		 11094008609030
  7601 		 1375025840 		 11093008609030 		 11094383634870
  7602 		 1375026140 		 11093383634870 		 11094758661010
  7603 		 1375026440 		 11093758661010 		 11095133687450
  7604 		 1375026740 		 11094133687450 		 11095508714190
  7605 		 1375027040 		 11094508714190 		 11095883741230
  7606 		 1375027340 		 11094883741230 		 11096258768570
  7607 		 1375027640 		 1109525

  7801 		 1375085846 		 11168019747619 		 11169394833465
  7802 		 1375086146 		 11168394833465 		 11169769919611
  7803 		 1375086446 		 11168769919611 		 11170145006057
  7804 		 1375086746 		 11169145006057 		 11170520092803
  7805 		 1375087046 		 11169520092803 		 11170895179849
  7806 		 1375087346 		 11169895179849 		 11171270267195
  7807 		 1375087646 		 11170270267195 		 11171645354841
  7808 		 1375087946 		 11170645354841 		 11172020442787
  7809 		 1375088246 		 11171020442787 		 11172395531033
  7810 		 1375088546 		 11171395531033 		 11172770619579
  7811 		 1375088846 		 11171770619579 		 11173145708425
  7812 		 1375089146 		 11172145708425 		 11173520797571
  7813 		 1375089446 		 11172520797571 		 11173895887017
  7814 		 1375089746 		 11172895887017 		 11174270976763
  7815 		 1375090046 		 11173270976763 		 11174646066809
  7816 		 1375090346 		 11173646066809 		 11175021157155
  7817 		 1375090646 		 11174021157155 		 11175396247801
  7818 		 1375090946 		 1117439

  8055 		 1375162052 		 11263301192575 		 11264676354627
  8056 		 1375162352 		 11263676354627 		 11265051516979
  8057 		 1375162652 		 11264051516979 		 11265426679631
  8058 		 1375162952 		 11264426679631 		 11265801842583
  8059 		 1375163252 		 11264801842583 		 11266177005835
  8060 		 1375163552 		 11265177005835 		 11266552169387
  8061 		 1375163852 		 11265552169387 		 11266927333239
  8062 		 1375164152 		 11265927333239 		 11267302497391
  8063 		 1375164452 		 11266302497391 		 11267677661843
  8064 		 1375164752 		 11266677661843 		 11268052826595
  8065 		 1375165052 		 11267052826595 		 11268427991647
  8066 		 1375165352 		 11267427991647 		 11268803156999
  8067 		 1375165653 		 11267803156999 		 11269178322652
  8068 		 1375165953 		 11268178322652 		 11269553488605
  8069 		 1375166253 		 11268553488605 		 11269928654858
  8070 		 1375166553 		 11268928654858 		 11270303821411
  8071 		 1375166853 		 11269303821411 		 11270678988264
  8072 		 1375167153 		 1126967

  8228 		 1375213956 		 11328208691353 		 11329583905309
  8229 		 1375214256 		 11328583905309 		 11329959119565
  8230 		 1375214556 		 11328959119565 		 11330334334121
  8231 		 1375214856 		 11329334334121 		 11330709548977
  8232 		 1375215156 		 11329709548977 		 11331084764133
  8233 		 1375215457 		 11330084764133 		 11331459979590
  8234 		 1375215757 		 11330459979590 		 11331835195347
  8235 		 1375216057 		 11330835195347 		 11332210411404
  8236 		 1375216357 		 11331210411404 		 11332585627761
  8237 		 1375216657 		 11331585627761 		 11332960844418
  8238 		 1375216957 		 11331960844418 		 11333336061375
  8239 		 1375217257 		 11332336061375 		 11333711278632
  8240 		 1375217557 		 11332711278632 		 11334086496189
  8241 		 1375217857 		 11333086496189 		 11334461714046
  8242 		 1375218157 		 11333461714046 		 11334836932203
  8243 		 1375218457 		 11333836932203 		 11335212150660
  8244 		 1375218757 		 11334212150660 		 11335587369417
  8245 		 1375219057 		 1133458